In [2]:
#Import Module
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import nltk
nltk.download('punkt')
nltk.download('stopwords')

import csv
import requests
from io import StringIO

#Module untuk Ekstraksi Feature
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Pemisahan data test dan train
from sklearn.model_selection import train_test_split

#Modul untuk pembuatan model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Import Data
df = pd.read_csv('/content/drive/MyDrive/Proyek 1_Dicoding/review_Ipunas.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12853 entries, 0 to 12852
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            12853 non-null  int64 
 1   reviewId              12853 non-null  object
 2   userName              12853 non-null  object
 3   userImage             12853 non-null  object
 4   content               12853 non-null  object
 5   score                 12853 non-null  int64 
 6   thumbsUpCount         12853 non-null  int64 
 7   reviewCreatedVersion  9791 non-null   object
 8   at                    12853 non-null  object
 9   replyContent          12466 non-null  object
 10  repliedAt             12466 non-null  object
 11  appVersion            9791 non-null   object
dtypes: int64(3), object(9)
memory usage: 1.2+ MB


In [4]:
# Membuat dataFrame baru dengan hanya mengambil kolom content saja
new_df = pd.DataFrame(df['content'])
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12853 entries, 0 to 12852
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  12853 non-null  object
dtypes: object(1)
memory usage: 100.5+ KB


In [5]:
# Membuat DataFrame baru (clean_df) dengan menghapus baris duplikat pada data
clean_df = new_df.drop_duplicates()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11822 entries, 0 to 12852
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  11822 non-null  object
dtypes: object(1)
memory usage: 184.7+ KB


In [6]:
#Text Preprocessing
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih",
              "wts": "jual", "wtt": "tukar", "bgt": "banget",
              "maks": "maksimal", "gak":"tidak", "ituh":"itu", "ga":"tidak",
              "apk":"aplikasi"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [7]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

In [8]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11822 entries, 0 to 12852
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   content               11822 non-null  object
 1   text_clean            11822 non-null  object
 2   text_casefoldingText  11822 non-null  object
 3   text_slangwords       11822 non-null  object
 4   text_tokenizingText   11822 non-null  object
 5   text_stopword         11822 non-null  object
 6   text_akhir            11822 non-null  object
dtypes: object(7)
memory usage: 738.9+ KB


In [9]:
#Memberikan Label pada Data
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [10]:
# Fungsi untuk menentukan polaritas sentimen dari review yang dibagi menjadi 3 kategori

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score > 0):
        polarity = 'positive'

    elif (score < 0):
        polarity = 'negative'

    else:
        polarity = 'neutral'

    return score, polarity

In [11]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
negative    5506
positive    4203
neutral     2113
Name: count, dtype: int64


In [12]:
clean_df.head(10)

,content,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity
0,Sejauh ini koleksi buku belum terlalu lengkap ...,Sejauh ini koleksi buku belum terlalu lengkap ...,sejauh ini koleksi buku belum terlalu lengkap ...,sejauh ini koleksi buku belum terlalu lengkap ...,"[sejauh, ini, koleksi, buku, belum, terlalu, l...","[koleksi, buku, lengkap, berdasarkan, genrepen...",koleksi buku lengkap berdasarkan genrepenulisn...,12,positive
1,Ini aplikasi bagus banget jujur (tolong naikin...,Ini aplikasi bagus banget jujur tolong naikin ...,ini aplikasi bagus banget jujur tolong naikin ...,ini aplikasi bagus banget jujur tolong naikin ...,"[ini, aplikasi, bagus, banget, jujur, tolong, ...","[aplikasi, bagus, banget, jujur, tolong, naiki...",aplikasi bagus banget jujur tolong naikin gaji...,-5,negative
2,"Sering bermasalah dgn sinyal, padahal sinyal s...",Sering bermasalah dgn sinyal padahal sinyal sa...,sering bermasalah dgn sinyal padahal sinyal sa...,sering bermasalah dgn sinyal padahal sinyal sa...,"[sering, bermasalah, dgn, sinyal, padahal, sin...","[bermasalah, dgn, sinyal, sinyal, tdk, loading...",bermasalah dgn sinyal sinyal tdk loading suka ...,-5,negative
3,"""Untuk Admin"" Aplikasi yg bermanfaat.. Dan sar...",Untuk Admin Aplikasi yg bermanfaat Dan saran s...,untuk admin aplikasi yg bermanfaat dan saran s...,untuk admin aplikasi yg bermanfaat dan saran s...,"[untuk, admin, aplikasi, yg, bermanfaat, dan, ...","[admin, aplikasi, yg, bermanfaat, saran, sy, u...",admin aplikasi yg bermanfaat saran sy utk ting...,0,neutral
4,"Apk nya sering ngebug, waktu mah login tiba-ti...",Apk nya sering ngebug waktu mah login tibatiba...,apk nya sering ngebug waktu mah login tibatiba...,aplikasi nya sering ngebug waktu mah login tib...,"[aplikasi, nya, sering, ngebug, waktu, mah, lo...","[aplikasi, ngebug, mah, login, tibatiba, kadan...",aplikasi ngebug mah login tibatiba kadang masu...,-15,negative
5,"Alhamdulillah, sudah banyak perkembangan seper...",Alhamdulillah sudah banyak perkembangan sepert...,alhamdulillah sudah banyak perkembangan sepert...,alhamdulillah sudah banyak perkembangan sepert...,"[alhamdulillah, sudah, banyak, perkembangan, s...","[alhamdulillah, perkembangan, highlight, tulis...",alhamdulillah perkembangan highlight tulisan s...,14,positive
6,"Aplikasi ini lumayan lah, awal awal nya kurang...",Aplikasi ini lumayan lah awal awal nya kurang ...,aplikasi ini lumayan lah awal awal nya kurang ...,aplikasi ini lumayan lah awal awal nya kurang ...,"[aplikasi, ini, lumayan, lah, awal, awal, nya,...","[aplikasi, lumayan, tertarik, buku, bersifat, ...",aplikasi lumayan tertarik buku bersifat non fi...,0,neutral
7,Suka banget! Tapi masih ada beberapa fitur yan...,Suka banget Tapi masih ada beberapa fitur yang...,suka banget tapi masih ada beberapa fitur yang...,suka banget tapi masih ada beberapa fitur yang...,"[suka, banget, tapi, masih, ada, beberapa, fit...","[suka, banget, fitur, berfungsi, salah, satuny...",suka banget fitur berfungsi salah satunya pena...,-5,negative
8,"Ipunas tuh bagus, dia punya potensi baik dan b...",Ipunas tuh bagus dia punya potensi baik dan be...,ipunas tuh bagus dia punya potensi baik dan be...,ipunas tuh bagus dia punya potensi baik dan be...,"[ipunas, tuh, bagus, dia, punya, potensi, baik...","[ipunas, tuh, bagus, potensi, bermanfaat, plea...",ipunas tuh bagus potensi bermanfaat pleaseeee ...,15,positive
9,"Ayolah pemerintah lebih urus ni aplikasi, ipus...",Ayolah pemerintah lebih urus ni aplikasi ipusn...,ayolah pemerintah lebih urus ni aplikasi ipusn...,ayolah pemerintah lebih urus ni aplikasi ipusn...,"[ayolah, pemerintah, lebih, urus, ni, aplikasi...","[ayolah, pemerintah, urus, ni, aplikasi, ipusn...",ayolah pemerintah urus ni aplikasi ipusnas pot...,-3,negative


In [14]:
#Ekstraksi Features
X = clean_df['text_akhir']
y = clean_df['polarity']

tokenizer = Tokenizer(num_words=20000, split=' ')
tokenizer.fit_on_texts(X.values)
X_seq = tokenizer.texts_to_sequences(X.values)
X_seq = pad_sequences(X_seq)

In [15]:
X_seq.shape

(11822, 104)

In [16]:
#Membuat model LSTM

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(20000, embed_dim))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [17]:
#Pembagian data training dan data testing
Y = pd.get_dummies(y).values
X_train, X_test, Y_train, Y_test = train_test_split(X_seq,Y, test_size = 0.3, random_state = 31)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(8275, 104) (8275, 3)
(3547, 104) (3547, 3)


In [18]:
#Training dan Evaluasi
batch_size = 32
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2, validation_data=(X_test, Y_test))

Epoch 1/5
259/259 - 93s - 358ms/step - accuracy: 0.6836 - loss: 0.7362 - val_accuracy: 0.8159 - val_loss: 0.4979
Epoch 2/5
259/259 - 139s - 537ms/step - accuracy: 0.8764 - loss: 0.3570 - val_accuracy: 0.8554 - val_loss: 0.4269
Epoch 3/5
259/259 - 86s - 332ms/step - accuracy: 0.9207 - loss: 0.2210 - val_accuracy: 0.8607 - val_loss: 0.3919
Epoch 4/5
259/259 - 141s - 546ms/step - accuracy: 0.9471 - loss: 0.1502 - val_accuracy: 0.8720 - val_loss: 0.4182
Epoch 5/5
259/259 - 140s - 541ms/step - accuracy: 0.9459 - loss: 0.1538 - val_accuracy: 0.8602 - val_loss: 0.4231


In [19]:
review = ['Aplikasinya keren banget. Sangat membantu menyelesaikan tugas skripsi pas gak bisa ke perpustakaan. Sangyang antrean bukunya lama banget']
review = tokenizer.texts_to_sequences(review)
review = pad_sequences(review)

sentiment = model.predict(review, batch_size=1,verbose = 2)[0]
print(sentiment)
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("neutral")
else :
    print('positif')



1/1 - 0s - 215ms/step
[0.01292498 0.00929568 0.97777927]
positif


In [1]:
!pip install pipreqs

In [20]:
!pipreqs '/content/drive/MyDrive/Proyek 1_Dicoding' --scan-notebooks

Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in /content/drive/MyDrive/Proyek 1_Dicoding/requirements.txt
